In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# 리뷰 텍스트 분석을 위한 추가 라이브러리
try:
    from wordcloud import WordCloud
    from collections import Counter
    import re
    print("✅ 워드클라우드 라이브러리 로딩 성공")
except ImportError:
    print("⚠️ 워드클라우드 라이브러리가 없습니다. 다음 명령어로 설치하세요:")
    print("!pip install wordcloud")

# 경고 메시지 무시
warnings.filterwarnings('ignore')

# 그래프 스타일 설정
plt.style.use('default')
sns.set_palette("husl")


In [ ]:
# CSV 파일 경로
facility_file = '전통시장_매칭데이터.csv'
review_file = 'review_result_test.csv'

# 데이터 불러오기 (한글 깨짐 방지를 위해 인코딩 처리)
try:
    df_facility = pd.read_csv(facility_file, encoding='cp949')
    df_review = pd.read_csv(review_file, encoding='cp949')
    print("cp949 인코딩으로 파일 로딩 성공")
except UnicodeDecodeError:
    print("cp949 인코딩 실패, utf-8으로 재시도")
    df_facility = pd.read_csv(facility_file, encoding='utf-8')
    df_review = pd.read_csv(review_file, encoding='utf-8')
    print("utf-8 인코딩으로 파일 로딩 성공")

print(f"시설 데이터: {df_facility.shape[0]}개 시장, {df_facility.shape[1]}개 컬럼")
print(f"리뷰 데이터: {df_review.shape[0]}개 시장, {df_review.shape[1]}개 컬럼")

# 시설 데이터의 중복 확인 및 제거
print(f"\n=== 시설 데이터 중복 확인 ===")
print(f"원본 시설 데이터 수: {len(df_facility)}")
print(f"고유 시장코드 수: {df_facility['시장코드'].nunique()}")

# 시장코드 중복 확인
duplicate_codes = df_facility[df_facility.duplicated(subset=['시장코드'], keep=False)]
if len(duplicate_codes) > 0:
    print(f"중복된 시장코드 발견: {len(duplicate_codes)}개")
    print("중복된 시장코드 상세 (상위 10개):")
    display(duplicate_codes[['시장코드', '시장명', '시군구', '지번주소']].sort_values('시장코드').head(10))
    
    # 중복된 시장코드 제거 (첫 번째만 유지)
    df_facility = df_facility.drop_duplicates(subset=['시장코드'], keep='first')
    print(f"\n중복 제거 후 시설 데이터 수: {len(df_facility)}")
else:
    print("중복된 시장코드 없음")


In [ ]:
print("=== 전통시장 시설 데이터 정보 ===")
print(df_facility.info())
print("\n=== 시설 데이터 상위 5개 ===")
display(df_facility.head())

print("\n=== 주요 시설 컬럼 확인 ===")
facility_related_cols = [col for col in df_facility.columns if col not in 
                        ['시장코드', '시장명', '지번주소', '도로명주소', '시도', '시군구', 
                         '시장개설주기', '점포수', '개설연도', '취급품목', '위도', '경도', '전화번호']]
print(f"시설 관련 컬럼 수: {len(facility_related_cols)}")
print("시설 컬럼 목록:")
for i, col in enumerate(facility_related_cols, 1):
    print(f"{i:2d}. {col}")


In [ ]:
print("=== 시장 리뷰 데이터 정보 ===")
print(df_review.info())
print("\n=== 리뷰 데이터 상위 5개 ===")
display(df_review.head())

print("\n=== 별점 데이터 현황 ===")
print(f"카카오 별점 보유 시장: {df_review['카카오별점'].notna().sum()}개")
print(f"네이버 별점 보유 시장: {df_review['네이버별점'].notna().sum()}개")
print(f"둘 중 하나라도 별점이 있는 시장: {(df_review['카카오별점'].notna() | df_review['네이버별점'].notna()).sum()}개")

print("\n=== 시장명 형식 비교 ===")
print("시설 데이터 시장명 예시:")
print(df_facility['시장명'].head(3).tolist())
print("\n리뷰 데이터 시장명 예시:")
print(df_review['시장명'].head(3).tolist())


In [ ]:
# 시설 데이터의 '시장명' 표준화 (지역명 추가 + 고유 식별자)
# 동일한 시군구 내 같은 이름의 시장을 구분하기 위해 고유 식별자 추가
df_facility['시장명_표준'] = df_facility['시군구'] + ' ' + df_facility['시장명']

# 중복된 시장명이 있는지 확인하고 고유 식별자 추가
duplicate_mask = df_facility.duplicated(subset=['시장명_표준'], keep=False)
if duplicate_mask.any():
    print(f"중복된 시장명 발견: {duplicate_mask.sum()}개")
    
    # 중복된 시장명에 대해 시장코드를 이용한 고유 식별자 추가
    df_facility['시장명_고유'] = df_facility['시장명_표준'].copy()
    for name in df_facility[duplicate_mask]['시장명_표준'].unique():
        mask = df_facility['시장명_표준'] == name
        indices = df_facility[mask].index
        for i, idx in enumerate(indices):
            if i > 0:  # 첫 번째는 그대로 두고, 두 번째부터 식별자 추가
                df_facility.loc[idx, '시장명_고유'] = f"{name}_{df_facility.loc[idx, '시장코드']}"
else:
    df_facility['시장명_고유'] = df_facility['시장명_표준']

print("=== 시장명 표준화 결과 ===")
comparison_df = pd.DataFrame({
    '원본 시장명': df_facility['시장명'].head(10),
    '표준화된 시장명': df_facility['시장명_표준'].head(10),
    '고유 시장명': df_facility['시장명_고유'].head(10),
    '시군구': df_facility['시군구'].head(10)
})
display(comparison_df)

print(f"\n시설 데이터 - 표준화 전 고유 시장명 수: {df_facility['시장명'].nunique()}")
print(f"시설 데이터 - 표준화 후 고유 시장명 수: {df_facility['시장명_표준'].nunique()}")
print(f"시설 데이터 - 고유 식별자 적용 후: {df_facility['시장명_고유'].nunique()}")
print(f"리뷰 데이터 - 고유 시장명 수: {df_review['시장명'].nunique()}")

# 중복 시장명 상세 확인
if duplicate_mask.any():
    print("\n=== 중복된 시장명 상세 정보 ===")
    duplicate_markets = df_facility[duplicate_mask][['시장코드', '시장명', '시군구', '시장명_표준', '시장명_고유']].sort_values('시장명_표준')
    display(duplicate_markets)


In [ ]:
# 시설 데이터를 기준으로 리뷰 데이터를 결합 (left join)
# 표준화된 시설 데이터의 시장명과 리뷰 데이터의 시장명을 매칭
df_merged = pd.merge(
    df_facility, 
    df_review, 
    left_on='시장명_표준', 
    right_on='시장명', 
    how='left'
)

print("=== 데이터 결합 결과 ===")
print(f"결합된 데이터 크기: {df_merged.shape}")
print(f"매칭된 시장 수: {df_merged['카카오별점'].notna().sum() + df_merged['네이버별점'].notna().sum() - df_merged[['카카오별점', '네이버별점']].notna().all(axis=1).sum()}")

# 결합 결과 확인
print("\n=== 결합된 데이터 상위 5개 ===")
display(df_merged[['시장명_x', '시장명_y', '카카오별점', '네이버별점']].head())

# 매칭 성공률 확인
total_facilities = len(df_facility)
matched_facilities = df_merged['카카오별점'].notna().sum() + df_merged['네이버별점'].notna().sum() - df_merged[['카카오별점', '네이버별점']].notna().all(axis=1).sum()
match_rate = (matched_facilities / total_facilities) * 100
print(f"\n매칭 성공률: {match_rate:.1f}% ({matched_facilities}/{total_facilities})")


In [ ]:
# 1. 종합만족도 점수 생성 (카카오, 네이버 별점 평균)
df_merged['종합만족도'] = df_merged[['카카오별점', '네이버별점']].mean(axis=1)

# 2. 분석할 주요 시설 컬럼 선택
facility_columns = [
    '아케이드', '주차장', '공중화장실', '고객휴게실', '고객동선통로',
    '종합콜센터', '유아놀이방', '엘리베이터/에스컬레이터', '문화교실', '시장전용고객주차장'
]

# 3. 시설 데이터(Y/N)를 숫자(1/0)로 변환
for col in facility_columns:
    if col in df_merged.columns:
        df_merged[col] = df_merged[col].apply(lambda x: 1 if x == 'Y' else 0)

print("=== 전처리 완료된 데이터 정보 ===")
selected_columns = ['시장명_x', '종합만족도'] + facility_columns
print(f"분석 대상 컬럼: {len(selected_columns)}개")
print(f"종합만족도가 있는 시장: {df_merged['종합만족도'].notna().sum()}개")

print("\n=== 전처리 완료된 데이터 샘플 ===")
display(df_merged[selected_columns].head(10))

# 4. 선택된 컬럼 데이터를 CSV 파일로 저장
try:
    df_merged[selected_columns].to_csv('전통시장_선택컬럼_데이터.csv', index=False, encoding='utf-8-sig')
    print(f"\n✅ '전통시장_선택컬럼_데이터.csv' 파일 저장 완료")
    print(f"📊 저장된 데이터: {len(df_merged)}개 시장, {len(selected_columns)}개 컬럼")
    print(f"📁 파일 위치: 현재 작업 디렉토리")
except Exception as e:
    print(f"❌ CSV 파일 저장 중 오류 발생: {e}")


In [ ]:
# 분석을 위해 별점이 있는 데이터만 추출
df_analysis = df_merged.dropna(subset=['종합만족도'])
print(f"=== 분석 대상 시장: {len(df_analysis)}개 ===")

# 종합만족도 기술통계
print("\n=== 종합만족도 기술통계 ===")
print(df_analysis['종합만족도'].describe())

# 시설별 보유 현황
print("\n=== 시설별 보유 현황 ===")
facility_stats = pd.DataFrame({
    '시설명': facility_columns,
    '보유시장수': [df_analysis[col].sum() for col in facility_columns],
    '보유비율(%)': [df_analysis[col].mean() * 100 for col in facility_columns]
})
facility_stats = facility_stats.sort_values('보유비율(%)', ascending=False)
display(facility_stats)

# 시설 보유 개수별 분포
df_analysis['총시설수'] = df_analysis[facility_columns].sum(axis=1)
print("\n=== 시장별 총 시설 보유 개수 분포 ===")
facility_count_dist = df_analysis['총시설수'].value_counts().sort_index()
print(facility_count_dist)


In [ ]:
# 상관관계 계산
corr_columns = ['종합만족도'] + facility_columns
correlation_matrix = df_analysis[corr_columns].corr()

print("=== 시설과 종합만족도의 상관관계 ===")
satisfaction_corr = correlation_matrix['종합만족도'].drop('종합만족도').sort_values(ascending=False)
print(satisfaction_corr)

# 상관관계 히트맵 시각화
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.3f', 
           square=True, linewidths=0.5, cbar_kws={"shrink": .8}, center=0)
plt.title('전통시장 시설과 종합만족도의 상관관계 분석', fontsize=16, pad=20, fontweight='bold')
plt.xlabel('시설 종류', fontsize=12)
plt.ylabel('시설 종류', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

print("\n=== 상관관계 해석 ===")
print("양의 상관관계(+): 해당 시설이 있을 때 만족도가 높아지는 경향")
print("음의 상관관계(-): 해당 시설이 있을 때 만족도가 낮아지는 경향")
print("0에 가까움: 해당 시설과 만족도 간 관계가 약함")


In [ ]:
# 시설 유무에 따른 만족도 비교
facility_impact = []
facility_impact_summary = []

for col in facility_columns:
    # 시설이 있는 경우와 없는 경우의 평균 만족도 계산
    mean_with = df_analysis[df_analysis[col] == 1]['종합만족도'].mean()
    mean_without = df_analysis[df_analysis[col] == 0]['종합만족도'].mean()
    difference = mean_with - mean_without
    
    # 시각화용 데이터
    facility_impact.append({'시설': col, '평균만족도': mean_with, '유무': '보유'})
    facility_impact.append({'시설': col, '평균만족도': mean_without, '유무': '미보유'})
    
    # 요약 통계
    facility_impact_summary.append({
        '시설': col,
        '보유시_평균만족도': round(mean_with, 3),
        '미보유시_평균만족도': round(mean_without, 3),
        '만족도_차이': round(difference, 3)
    })

# 요약 테이블 출력
print("=== 시설 유무에 따른 만족도 차이 분석 ===")
summary_df = pd.DataFrame(facility_impact_summary)
summary_df = summary_df.sort_values('만족도_차이', ascending=False)
display(summary_df)

# 시각화
df_impact = pd.DataFrame(facility_impact)
plt.figure(figsize=(14, 8))
sns.barplot(x='시설', y='평균만족도', hue='유무', data=df_impact)
plt.title('시설 유무에 따른 시장 평균 만족도 비교', fontsize=16, pad=20)
plt.ylabel('평균 만족도 점수', fontsize=12)
plt.xlabel('시설 종류', fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend(title='시설 보유 여부', fontsize=10)
plt.tight_layout()
plt.show()


In [ ]:
# 총 시설 개수와 만족도의 관계 분석
facility_count_satisfaction = df_analysis.groupby('총시설수')['종합만족도'].agg(['mean', 'count']).reset_index()
facility_count_satisfaction.columns = ['총시설수', '평균만족도', '시장수']

print("=== 총 시설 개수별 평균 만족도 ===")
display(facility_count_satisfaction)

# 총 시설 개수와 만족도의 상관관계
total_facility_corr = df_analysis[['총시설수', '종합만족도']].corr().iloc[0, 1]
print(f"\n총 시설 개수와 종합만족도의 상관관계: {total_facility_corr:.3f}")

# 시각화
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# 산점도
ax1.scatter(df_analysis['총시설수'], df_analysis['종합만족도'], alpha=0.6)
ax1.set_xlabel('총 시설 개수')
ax1.set_ylabel('종합만족도')
ax1.set_title('총 시설 개수 vs 종합만족도')
ax1.grid(True, alpha=0.3)

# 평균 만족도 막대그래프
ax2.bar(facility_count_satisfaction['총시설수'], facility_count_satisfaction['평균만족도'])
ax2.set_xlabel('총 시설 개수')
ax2.set_ylabel('평균 만족도')
ax2.set_title('총 시설 개수별 평균 만족도')
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# 주요 발견사항 정리
print("=== 🔍 주요 발견사항 ===")
print()

# 1. 가장 만족도에 긍정적 영향을 미치는 시설
top_positive_facility = satisfaction_corr.idxmax()
top_positive_corr = satisfaction_corr.max()
print(f"1️⃣ 만족도에 가장 긍정적 영향을 미치는 시설: {top_positive_facility}")
print(f"   상관계수: {top_positive_corr:.3f}")

# 2. 가장 만족도 차이를 만드는 시설
max_diff_facility = summary_df.iloc[0]['시설']
max_diff_value = summary_df.iloc[0]['만족도_차이']
print(f"\n2️⃣ 보유 여부에 따른 만족도 차이가 가장 큰 시설: {max_diff_facility}")
print(f"   만족도 차이: {max_diff_value:.3f}점")

# 3. 총 시설 개수와 만족도의 관계
print(f"\n3️⃣ 총 시설 개수와 만족도의 상관관계: {total_facility_corr:.3f}")
if total_facility_corr > 0.1:
    print("   → 시설이 많을수록 만족도가 높아지는 경향")
elif total_facility_corr < -0.1:
    print("   → 시설이 많을수록 만족도가 낮아지는 경향")
else:
    print("   → 시설 개수와 만족도 간 뚜렷한 관계 없음")

# 4. 시설 보유율이 낮으면서 만족도 영향이 큰 시설 (개선 우선순위)
facility_stats_with_impact = facility_stats.merge(
    summary_df[['시설', '만족도_차이']], 
    left_on='시설명', 
    right_on='시설'
)
facility_stats_with_impact['개선우선도'] = (
    facility_stats_with_impact['만족도_차이'] * 
    (100 - facility_stats_with_impact['보유비율(%)']) / 100
)
priority_facilities = facility_stats_with_impact.sort_values('개선우선도', ascending=False)

print(f"\n4️⃣ 시설 개선 우선순위 (보유율이 낮으면서 만족도 영향이 큰 시설):")
for i, row in priority_facilities.head(3).iterrows():
    print(f"   {row.name+1}위: {row['시설명']} (보유율: {row['보유비율(%)']:.1f}%, 만족도차이: {row['만족도_차이']:.3f})")

print(f"\n5️⃣ 분석 대상 시장 현황:")
print(f"   전체 시장: {len(df_merged)}개")
print(f"   리뷰 데이터 보유 시장: {len(df_analysis)}개")
print(f"   매칭 성공률: {(len(df_analysis)/len(df_facility)*100):.1f}%")
print(f"   평균 종합만족도: {df_analysis['종합만족도'].mean():.2f}점")


In [ ]:
# 리뷰 텍스트 데이터 확인
print("=== 리뷰 텍스트 데이터 현황 ===")
print(f"전체 리뷰 데이터: {len(df_review)}개")

# 카카오와 네이버 리뷰 텍스트 컬럼 확인
review_columns = [col for col in df_review.columns if '리뷰' in col or 'review' in col.lower()]
print(f"리뷰 관련 컬럼: {review_columns}")

# 리뷰 텍스트가 있는 컬럼 찾기
text_columns = []
for col in df_review.columns:
    if df_review[col].dtype == 'object' and col not in ['시장명']:
        # 텍스트 길이가 긴 컬럼들을 리뷰 텍스트로 간주
        avg_length = df_review[col].dropna().astype(str).str.len().mean()
        if avg_length > 10:  # 평균 길이가 10자 이상인 컬럼
            text_columns.append(col)
            print(f"텍스트 컬럼 발견: {col} (평균 길이: {avg_length:.1f}자)")

print(f"\n분석 대상 텍스트 컬럼: {text_columns}")

# 리뷰 텍스트 샘플 확인
if text_columns:
    print(f"\n=== {text_columns[0]} 샘플 ===")
    sample_reviews = df_review[text_columns[0]].dropna().head(5)
    for i, review in enumerate(sample_reviews, 1):
        print(f"{i}. {str(review)[:100]}...")
else:
    print("⚠️ 리뷰 텍스트 컬럼을 찾을 수 없습니다.")


In [ ]:
# 25개 주요 시설에 대한 포괄적 키워드 매핑 정의
# 고객 리뷰에서 실제로 사용되는 다양한 표현들을 포함
facility_keywords_mapping = {
    '아케이드': ['아케이드', '아케이드형', '실내', '지붕', '덮개', '비가림', '천장', '차양', '햇빛', '그늘', '비오는날', '날씨'],
    '엘리베이터/에스컬레이터': ['엘리베이터', '에스컬레이터', '승강기', '계단', '층', '올라가', '내려가', '위아래', '오르내리', '무거운짐', '계단힘들'],
    '고객지원센터': ['고객지원', '안내센터', '안내소', '정보센터', '도움', '문의', '상담', '서비스센터', '고객센터', '안내데스크', '정보'],
    '스프링쿨러': ['스프링쿨러', '소화', '화재', '안전', '방화', '소화기', '화재예방', '안전시설', '소화설비'],
    '화재감지기': ['화재감지기', '감지기', '화재', '안전', '경보', '화재경보', '연기감지', '알람', '안전장치'],
    '유아놀이방': ['유아놀이방', '놀이방', '아이', '유아', '어린이', '키즈', '놀이', '놀이터', '아기', '육아', '아이들', '어린이시설'],
    '종합콜센터': ['콜센터', '전화', '연락', '상담', '통화', '전화상담', '콜', '전화문의'],
    '고객휴게실': ['휴게실', '휴게', '쉬는', '앉을', '의자', '벤치', '쉴', '쉼터', '앉는곳', '편의시설', '쉴곳', '휴식공간'],
    '수유센터': ['수유센터', '수유', '수유실', '아기', '젖먹이', '모유', '수유공간', '육아', '수유시설'],
    '물품보관함': ['보관함', '물품보관', '짐보관', '보관', '맡기', '사물함', '보관소', '짐맡기', '가방보관'],
    '자전거보관함': ['자전거보관', '자전거', '자전거주차', '자전거보관함', '바이크', '자전거거치', '자전거보관소', '자전거주차장'],
    '체육시설': ['체육시설', '운동', '체육', '헬스', '피트니스', '운동시설', '체육관', '운동공간', '헬스장'],
    '간이도서관': ['도서관', '책', '독서', '도서', '서적', '읽기', '도서실', '책읽기', '독서공간'],
    '쇼핑카트': ['쇼핑카트', '카트', '수레', '끌고', '밀고', '장보기', '쇼핑수레', '장바구니', '카트이용'],
    '외국인안내센터': ['외국인안내', '외국인', '영어', '통역', '번역', '외국어', '국제', '관광안내', '외국인서비스', '영어안내'],
    '고객동선통로': ['통로', '동선', '길', '이동', '다니기', '걷기', '이동통로', '보행', '동선설계', '이동편의', '길찾기'],
    '방송센터': ['방송', '안내방송', '스피커', '음성', '방송시설', '음향', '알림', '방송안내', '음성안내'],
    '문화교실': ['문화교실', '교실', '강의', '수업', '배우', '문화센터', '강좌', '교육프로그램', '문화활동', '배움'],
    '공동물류창고': ['창고', '물류', '보관', '저장', '물류센터', '창고시설', '보관창고', '물류시설'],
    '시장전용고객주차장': ['주차장', '주차', '차', '자동차', '주차공간', '주차비', '전용주차장', '고객주차', '주차시설', '주차편의'],
    '교육장': ['교육장', '교육', '강의실', '세미나', '교육시설', '강의', '교육공간', '교육프로그램'],
    '회의실': ['회의실', '회의', '미팅', '회의공간', '모임', '상담실', '회의시설'],
    '자동심장충격기': ['자동심장충격기', 'AED', '응급', '심장', '응급처치', '의료기기', '구급', '응급의료', '심폐소생'],
    '공중화장실': ['화장실', '변소', 'toilet', '화장실이', '볼일', '화장실시설', '위생시설', '편의시설', '화장실깨끗', '화장실더러'],
    '주차장': ['주차장', '주차', '차', '자동차', '주차공간', '주차비', '주차하기', '주차시설', '차량', '주차편의', '주차문제']
}

def extract_facility_keywords(text_series):
    """리뷰에서 시설 관련 키워드 추출 - 향상된 버전"""
    # 모든 텍스트 합치기
    all_text = ' '.join(text_series.dropna().astype(str))
    
    # 텍스트 전처리 (더 정교한 전처리)
    text = str(all_text).lower()
    # 특수문자 제거하되 한글, 영어, 숫자, 공백만 유지
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', ' ', text)
    # 연속된 공백을 하나로 통일
    text = re.sub(r'\s+', ' ', text)
    # 앞뒤 공백 제거
    text = text.strip()
    
    facility_counts = {}
    
    # 각 시설별로 관련 키워드 검색
    for facility, keywords in facility_keywords_mapping.items():
        total_count = 0
        matched_keywords = []
        keyword_details = {}
        
        for keyword in keywords:
            keyword_lower = keyword.lower()
            
            # 정확한 단어 매칭을 위해 단어 경계 고려
            # 단순 포함 검색과 단어 경계 검색 모두 사용
            simple_count = text.count(keyword_lower)
            
            # 단어 경계를 고려한 검색 (더 정확한 매칭)
            word_pattern = r'\b' + re.escape(keyword_lower) + r'\b'
            word_matches = len(re.findall(word_pattern, text))
            
            # 두 방법 중 더 높은 값 사용 (한글의 경우 단어 경계가 명확하지 않을 수 있음)
            keyword_count = max(simple_count, word_matches)
            
            if keyword_count > 0:
                total_count += keyword_count
                matched_keywords.append(f"{keyword}({keyword_count})")
                keyword_details[keyword] = keyword_count
        
        if total_count > 0:
            facility_counts[facility] = {
                'count': total_count,
                'matched_keywords': matched_keywords,
                'keyword_details': keyword_details,
                'unique_keywords': len(keyword_details)  # 매칭된 고유 키워드 수
            }
    
    return facility_counts

# 리뷰 텍스트가 있는 경우에만 분석 진행
if text_columns:
    print("=== 25개 주요 시설 관련 키워드 분석 ===")
    print("전통시장 고객들이 리뷰에서 언급하는 시설별 관심도를 분석합니다.")
    print("이를 통해 실제 고객 니즈와 시설 투자 우선순위를 파악할 수 있습니다.\n")
    
    # 모든 텍스트 컬럼의 데이터를 합쳐서 분석
    all_reviews = pd.Series()
    for col in text_columns:
        reviews = df_review[col].dropna()
        all_reviews = pd.concat([all_reviews, reviews], ignore_index=True)
    
    total_reviews = len(all_reviews)
    print(f"📊 분석 대상 리뷰 수: {total_reviews:,}개")
    print(f"🎯 분석 대상 시설: {len(facility_keywords_mapping)}개")
    
    # 시설 관련 키워드 추출
    facility_keyword_results = extract_facility_keywords(all_reviews)
    
    if facility_keyword_results:
        # 결과를 빈도순으로 정렬
        sorted_facilities = sorted(facility_keyword_results.items(), 
                                 key=lambda x: x[1]['count'], reverse=True)
        
        print(f"\n=== 🏆 리뷰에서 언급된 시설 순위 (총 {len(sorted_facilities)}개) ===")
        print("순위 | 시설명 | 언급횟수 | 주요키워드")
        print("-" * 100)
        
        for i, (facility, data) in enumerate(sorted_facilities, 1):
            main_keywords = ', '.join([kw.split('(')[0] for kw in data['matched_keywords'][:3]])
            print(f"{i:2d}위 | {facility:<15} | {data['count']:4d}회 | {main_keywords}")
        
        # 상세 데이터프레임 생성
        facility_keywords_df = pd.DataFrame([
            {
                '순위': i,
                '시설명': facility, 
                '언급횟수': data['count'],
                '고유키워드수': data['unique_keywords'],
                '관련키워드': ', '.join(data['matched_keywords'][:5])  # 상위 5개 표시
            }
            for i, (facility, data) in enumerate(sorted_facilities, 1)
        ])
        
        # 전체 시설 목록과 비교
        all_facilities = list(facility_keywords_mapping.keys())
        mentioned_facilities = [facility for facility, _ in sorted_facilities]
        not_mentioned = [f for f in all_facilities if f not in mentioned_facilities]
        
        print(f"\n• 리뷰에서 언급된 시설: {len(mentioned_facilities)}개 ({len(mentioned_facilities)/len(all_facilities)*100:.1f}%)")
        print(f"• 리뷰에서 언급되지 않은 시설: {len(not_mentioned)}개")
        
        if not_mentioned:
            print(f"\n⚠️ 언급되지 않은 시설들:")
            for facility in not_mentioned:
                print(f"   - {facility}")
            print("\n💡 이러한 시설들은 고객 인지도가 낮거나 실제 필요성이 낮을 수 있습니다.")
        
    else:
        print("⚠️ 리뷰에서 시설 관련 키워드를 찾을 수 없습니다.")
        facility_keywords_df = pd.DataFrame()
        sorted_facilities = []
    
else:
    print("⚠️ 분석할 리뷰 텍스트가 없습니다.")
    facility_keywords_df = pd.DataFrame()
    sorted_facilities = []


In [ ]:
# 시설 관련 워드클라우드 생성 및 시각화
if sorted_facilities and len(sorted_facilities) > 0:
    try:
        # 워드클라우드 생성을 위한 딕셔너리 형태로 변환
        facility_freq = {facility: data['count'] for facility, data in sorted_facilities}
        
        # 워드클라우드 설정
        wordcloud = WordCloud(
            font_path=None,  # 시스템 기본 폰트 사용
            width=1000, 
            height=600,
            background_color='white',
            max_words=len(facility_freq),
            colormap='Set3',
            relative_scaling=0.6,
            random_state=42
        ).generate_from_frequencies(facility_freq)
        
        # 시각화
        plt.figure(figsize=(14, 8))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.title('전통시장 리뷰에서 언급된 시설 워드클라우드\n(크기는 언급 빈도에 비례)', fontsize=16, pad=20)
        plt.tight_layout()
        plt.show()
        
        print("✅ 시설 관련 워드클라우드 생성 완료")
        
        # 시설별 언급 빈도 막대그래프
        plt.figure(figsize=(14, 8))
        
        facilities = [facility for facility, _ in sorted_facilities]
        counts = [data['count'] for _, data in sorted_facilities]
        
        bars = plt.barh(range(len(facilities)), counts, color='skyblue', alpha=0.8)
        plt.yticks(range(len(facilities)), facilities)
        plt.xlabel('리뷰 언급 횟수', fontsize=12)
        plt.title('전통시장 시설별 리뷰 언급 빈도\n(고객들이 중요하게 생각하는 시설 순위)', fontsize=16, pad=20)
        plt.gca().invert_yaxis()  # 상위 시설이 위에 오도록
        
        # 막대에 빈도 수치 표시
        for i, bar in enumerate(bars):
            plt.text(bar.get_width() + max(counts)*0.01, bar.get_y() + bar.get_height()/2, 
                     f'{counts[i]}회', 
                     va='center', fontsize=10, fontweight='bold')
        
        plt.grid(axis='x', alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        # 상세 분석 테이블 출력
        print("\n=== 시설별 리뷰 언급 상세 분석 ===")
        display(facility_keywords_df)
        
    except Exception as e:
        print(f"⚠️ 워드클라우드 생성 중 오류 발생: {e}")
        print("대신 시설별 언급 빈도 분석 결과를 표시합니다.")
        
        # 시설별 언급 빈도 테이블 출력
        print("\n=== 시설별 리뷰 언급 빈도 ===")
        if not facility_keywords_df.empty:
            display(facility_keywords_df)
        else:
            print("분석할 데이터가 없습니다.")

else:
    print("⚠️ 리뷰에서 시설 관련 키워드를 찾을 수 없어 워드클라우드를 생성할 수 없습니다.")


In [ ]:
# 워드클라우드 생성 및 시각화
if keywords and len(keywords) > 0:
    try:
        # 워드클라우드 생성을 위한 딕셔너리 형태로 변환
        word_freq = dict(keywords[:30])  # 상위 30개 키워드만 사용
        
        # 워드클라우드 설정
        wordcloud = WordCloud(
            font_path=None,  # 시스템 기본 폰트 사용
            width=800, 
            height=600,
            background_color='white',
            max_words=30,
            colormap='viridis',
            relative_scaling=0.5,
            random_state=42
        ).generate_from_frequencies(word_freq)
        
        # 시각화
        plt.figure(figsize=(12, 8))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.title('전통시장 리뷰 키워드 워드클라우드', fontsize=16, pad=20)
        plt.tight_layout()
        plt.show()
        
        print("✅ 워드클라우드 생성 완료")
        
        # 키워드 빈도 막대그래프
        plt.figure(figsize=(12, 8))
        top_keywords = keywords_df.head(15)
        
        bars = plt.barh(range(len(top_keywords)), top_keywords['빈도'])
        plt.yticks(range(len(top_keywords)), top_keywords['키워드'])
        plt.xlabel('빈도')
        plt.title('전통시장 리뷰 상위 키워드 빈도', fontsize=16)
        plt.gca().invert_yaxis()  # 상위 키워드가 위에 오도록
        
        # 막대에 빈도 수치 표시
        for i, bar in enumerate(bars):
            plt.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2, 
                     f'{top_keywords.iloc[i]["빈도"]}', 
                     va='center', fontsize=10)
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"⚠️ 워드클라우드 생성 중 오류 발생: {e}")
        print("대신 키워드 빈도 분석 결과를 표시합니다.")
        
        # 키워드 빈도 테이블 출력
        print("\n=== 상위 키워드 빈도 분석 ===")
        display(keywords_df.head(20))

else:
    print("⚠️ 워드클라우드를 생성할 키워드가 없습니다.")


In [ ]:
# 시설별 고객 관심도 vs 실제 보유율 비교 분석 (핵심 정책 결정 도구)
if sorted_facilities and len(sorted_facilities) > 0:
    print("=== 🎯 고객 관심도 vs 실제 보유율 비교 분석 ===")
    print("이 분석은 고객이 원하는 시설과 실제 보유 현황의 격차를 파악하여")
    print("투자 우선순위를 결정하는 핵심 정책 도구입니다.\n")
    
    # 시설별 보유율 계산 (분석 대상 시장 기준)
    facility_ownership = {}
    available_facilities = []
    
    for facility in facility_keywords_mapping.keys():
        if facility in df_analysis.columns:
            ownership_rate = df_analysis[facility].mean() * 100
            facility_ownership[facility] = ownership_rate
            available_facilities.append(facility)
    
    print(f"📊 보유율 데이터가 있는 시설: {len(available_facilities)}개")
    print(f"📊 리뷰에서 언급된 시설: {len(sorted_facilities)}개")
    
    # 비교 분석 데이터 생성
    comparison_data = []
    for facility, data in sorted_facilities:
        mention_count = data['count']
        importance_score = data['importance_score']
        mention_rate = data['mention_rate']
        ownership_rate = facility_ownership.get(facility, 0)
        
        # 관심도-보유율 격차 계산
        interest_ownership_gap = mention_rate - (ownership_rate / 10)  # 스케일 조정
        
        comparison_data.append({
            '시설명': facility,
            '리뷰언급횟수': mention_count,
            '중요도점수': importance_score,
            '리뷰언급률(%)': mention_rate,
            '실제보유율(%)': round(ownership_rate, 1),
            '관심도순위': sorted_facilities.index((facility, data)) + 1,
            '관심도보유율격차': round(interest_ownership_gap, 2),
            '투자우선도': '최우선' if (mention_rate > np.median([d['mention_rate'] for _, d in sorted_facilities]) and ownership_rate < np.median([facility_ownership.get(f, 0) for f, _ in sorted_facilities])) else '고려대상' if mention_rate > np.median([d['mention_rate'] for _, d in sorted_facilities]) else '보류',
            '관련키워드': ', '.join(data['matched_keywords'][:3])
        })
    
    # 데이터프레임으로 변환
    comparison_df = pd.DataFrame(comparison_data)
    
    print("\n=== 📋 고객 관심도 vs 실제 보유율 종합 비교표 ===")
    display(comparison_df.sort_values('관심도순위'))
    
    # 투자 우선순위 분석
    print("\n=== 🚀 투자 우선순위 분석 결과 ===")
    
    # 최우선 투자 대상 시설
    top_priority = comparison_df[comparison_df['투자우선도'] == '최우선'].sort_values('관심도순위')
    consider_priority = comparison_df[comparison_df['투자우선도'] == '고려대상'].sort_values('관심도순위')
    
    if not top_priority.empty:
        print("🔥 최우선 투자 대상 시설 (고관심도 + 저보유율):")
        for idx, row in top_priority.iterrows():
            print(f"   • {row['시설명']}: 관심도 {row['관심도순위']}위, 보유율 {row['실제보유율(%)']}%")
            print(f"     → 리뷰 언급 {row['리뷰언급횟수']}회 (언급률 {row['리뷰언급률(%)']}%)")
            print(f"     → ROI 예상: 높음 (고객 니즈 대비 공급 부족)")
            print()
    else:
        print("🔥 최우선 투자 대상 시설: 없음")
    
    if not consider_priority.empty:
        print("⚡ 2차 투자 고려 시설 (고관심도):")
        for idx, row in consider_priority.head(5).iterrows():  # 상위 5개만
            print(f"   • {row['시설명']}: 관심도 {row['관심도순위']}위, 보유율 {row['실제보유율(%)']}%")
            print(f"     → 리뷰 언급 {row['리뷰언급횟수']}회")
        print()
    
    # 시각화: 관심도 vs 보유율 산점도 (투자 우선순위 매트릭스)
    plt.figure(figsize=(16, 12))
    
    # 데이터 준비
    x_values = comparison_df['실제보유율(%)']
    y_values = comparison_df['리뷰언급률(%)']
    sizes = comparison_df['리뷰언급횟수'] * 3  # 크기 조정
    
    # 투자우선도별 색상 매핑
    priority_colors = {'최우선': '#FF4444', '고려대상': '#FFA500', '보류': '#90EE90'}
    colors = [priority_colors[priority] for priority in comparison_df['투자우선도']]
    
    scatter = plt.scatter(x_values, y_values, s=sizes, c=colors, alpha=0.7, 
                         edgecolors='black', linewidth=1.5)
    
    # 시설명 라벨 추가
    for idx, row in comparison_df.iterrows():
        plt.annotate(row['시설명'], 
                    (row['실제보유율(%)'], row['리뷰언급률(%)']),
                    xytext=(5, 5), textcoords='offset points',
                    fontsize=10, fontweight='bold', alpha=0.9)
    
    # 기준선 추가 (중간값 기준)
    median_ownership = comparison_df['실제보유율(%)'].median()
    median_mention = comparison_df['리뷰언급률(%)'].median()
    
    plt.axhline(y=median_mention, color='red', linestyle='--', alpha=0.6, linewidth=2, label=f'평균 언급률 ({median_mention:.2f}%)')
    plt.axvline(x=median_ownership, color='red', linestyle='--', alpha=0.6, linewidth=2, label=f'평균 보유율 ({median_ownership:.1f}%)')
    
    # 사분면 라벨 추가
    plt.text(median_ownership * 0.3, median_mention * 1.7, '고관심도\n저보유율\n(최우선 투자)', 
             fontsize=12, fontweight='bold', ha='center', va='center',
             bbox=dict(boxstyle="round,pad=0.3", facecolor='#FFE6E6', alpha=0.8))
    
    plt.text(median_ownership * 1.7, median_mention * 1.7, '고관심도\n고보유율\n(유지관리)', 
             fontsize=12, fontweight='bold', ha='center', va='center',
             bbox=dict(boxstyle="round,pad=0.3", facecolor='#E6FFE6', alpha=0.8))
    
    plt.text(median_ownership * 0.3, median_mention * 0.3, '저관심도\n저보유율\n(투자보류)', 
             fontsize=12, fontweight='bold', ha='center', va='center',
             bbox=dict(boxstyle="round,pad=0.3", facecolor='#F0F0F0', alpha=0.8))
    
    plt.text(median_ownership * 1.7, median_mention * 0.3, '저관심도\n고보유율\n(재검토필요)', 
             fontsize=12, fontweight='bold', ha='center', va='center',
             bbox=dict(boxstyle="round,pad=0.3", facecolor='#FFF0E6', alpha=0.8))
    
    plt.xlabel('실제 보유율 (%)', fontsize=14, fontweight='bold')
    plt.ylabel('리뷰 언급률 (%)', fontsize=14, fontweight='bold')
    plt.title('🎯 시설 투자 우선순위 매트릭스\n(고객 관심도 vs 실제 보유율 비교)', 
              fontsize=18, pad=30, fontweight='bold')
    
    # 범례 추가
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor=priority_colors['최우선'], label='최우선 투자'),
        Patch(facecolor=priority_colors['고려대상'], label='2차 투자 고려'),
        Patch(facecolor=priority_colors['보류'], label='투자 보류'),
        plt.Line2D([0], [0], color='red', linestyle='--', alpha=0.6, label='기준선 (중간값)')
    ]
    plt.legend(handles=legend_elements, loc='upper left', fontsize=12, framealpha=0.9)
    
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # 투자 효과 예측 분석
    print("=== 💰 투자 효과 예측 분석 ===")
    if not top_priority.empty:
        total_mentions = top_priority['리뷰언급횟수'].sum()
        avg_ownership = top_priority['실제보유율(%)'].mean()
        
        print(f"최우선 투자 시설들의 총 리뷰 언급: {total_mentions}회")
        print(f"현재 평균 보유율: {avg_ownership:.1f}%")
        print(f"예상 투자 효과: 높음 (고객 만족도 직접적 개선 예상)")
        print(f"권장 투자 순서: 리뷰 언급 빈도 순")
        
        print("\n📈 단계별 투자 계획 제안:")
        for i, (idx, row) in enumerate(top_priority.iterrows(), 1):
            print(f"   {i}단계: {row['시설명']} (언급 {row['리뷰언급횟수']}회, 현재 보유율 {row['실제보유율(%)']}%)")
    
else:
    print("⚠️ 비교 분석을 위한 데이터가 부족합니다.")
    comparison_df = pd.DataFrame()
    plt.figure(figsize=(12, 8))
    scatter = plt.scatter(comparison_df['실제보유율(%)'], comparison_df['리뷰언급횟수'], 
                         s=100, alpha=0.7, c=comparison_df['관심도순위'], cmap='viridis_r')
    
    # 시설명 라벨 추가
    for idx, row in comparison_df.iterrows():
        plt.annotate(row['시설명'], 
                    (row['실제보유율(%)'], row['리뷰언급횟수']),
                    xytext=(5, 5), textcoords='offset points',
                    fontsize=9, alpha=0.8)
    
    plt.xlabel('실제 보유율 (%)', fontsize=12)
    plt.ylabel('리뷰 언급 횟수', fontsize=12)
    plt.title('전통시장 시설별 고객 관심도 vs 실제 보유율\n(우상단: 관심도↑보유율↑, 좌상단: 관심도↑보유율↓)', 
              fontsize=14, pad=20)
    
    # 기준선 추가
    plt.axhline(y=comparison_df['리뷰언급횟수'].median(), color='red', linestyle='--', alpha=0.5, label='언급횟수 중간값')
    plt.axvline(x=comparison_df['실제보유율(%)'].median(), color='red', linestyle='--', alpha=0.5, label='보유율 중간값')
    
    plt.colorbar(scatter, label='관심도 순위 (낮을수록 관심도 높음)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # 정책 제언을 위한 상위 시설 저장
    top_mentioned_facilities = [facility for facility, _ in sorted_facilities[:5]]
    
else:
    print("⚠️ 시설 관련 분석 결과가 없어 비교 분석을 수행할 수 없습니다.")
    top_mentioned_facilities = []
    comparison_df = pd.DataFrame()


In [ ]:
# 키워드 카테고리 분석
if keywords and len(keywords) > 0:
    print("=== 리뷰 키워드 카테고리 분석 ===")
    
    # 키워드를 카테고리별로 분류
    categories = {
        '접근성/교통': ['주차', '교통', '버스', '지하철', '역', '접근', '찾기', '위치', '거리', '가까운', '멀다'],
        '가격/경제성': ['가격', '비싸', '싸다', '저렴', '비용', '돈', '값', '경제', '할인', '세일'],
        '품질/신선도': ['신선', '맛', '품질', '좋다', '나쁘다', '맛있', '맛없', '질', '상태', '깨끗'],
        '시설/편의성': ['시설', '화장실', '휴게', '편의', '깨끗', '더럽', '낡은', '새로운', '현대적'],
        '서비스/친절': ['친절', '불친절', '서비스', '응대', '직원', '사장', '상인', '태도'],
        '다양성/선택': ['다양', '종류', '선택', '많다', '적다', '풍부', '부족', '여러', '가지'],
        '분위기/환경': ['분위기', '환경', '조용', '시끄럽', '북적', '활기', '전통', '옛날', '정겨운']
    }
    
    # 각 카테고리별 키워드 매칭
    category_scores = {}
    top_keywords_list = [word for word, count in keywords[:30]]
    
    for category, category_keywords in categories.items():
        score = 0
        matched_keywords = []
        
        for keyword in top_keywords_list:
            for cat_keyword in category_keywords:
                if cat_keyword in keyword or keyword in cat_keyword:
                    # 해당 키워드의 빈도를 점수에 추가
                    keyword_freq = next((count for word, count in keywords if word == keyword), 0)
                    score += keyword_freq
                    matched_keywords.append(f"{keyword}({keyword_freq})")
                    break
        
        if matched_keywords:
            category_scores[category] = {
                'score': score,
                'keywords': matched_keywords
            }
    
    # 카테고리별 점수 정렬
    sorted_categories = sorted(category_scores.items(), key=lambda x: x[1]['score'], reverse=True)
    
    print("고객들이 가장 많이 언급하는 관심사 (카테고리별):")
    for i, (category, data) in enumerate(sorted_categories, 1):
        print(f"{i}. {category}: {data['score']}점")
        print(f"   관련 키워드: {', '.join(data['keywords'][:5])}")  # 상위 5개만 표시
        print()
    
    # 상위 3개 카테고리 저장 (정책 제언용)
    top_categories = [cat for cat, _ in sorted_categories[:3]]
    
else:
    print("⚠️ 키워드 분석 결과가 없어 카테고리 분석을 수행할 수 없습니다.")
    top_categories = []


In [ ]:
# 만족도가 낮으면서 주요 시설이 부족한 시장들 식별
low_satisfaction_threshold = df_analysis['종합만족도'].quantile(0.3)  # 하위 30%
high_impact_facilities = satisfaction_corr.nlargest(3).index.tolist()  # 상위 3개 고효과 시설

print(f"=== 개선이 필요한 시장 식별 기준 ===")
print(f"만족도 기준: {low_satisfaction_threshold:.2f}점 이하 (하위 30%)")
print(f"주요 시설: {', '.join(high_impact_facilities)}")

# 조건에 맞는 시장 추출
improvement_needed = df_analysis[
    (df_analysis['종합만족도'] <= low_satisfaction_threshold) &
    (df_analysis[high_impact_facilities].sum(axis=1) < len(high_impact_facilities))
].copy()

improvement_needed['부족한_주요시설수'] = len(high_impact_facilities) - improvement_needed[high_impact_facilities].sum(axis=1)
improvement_needed_sorted = improvement_needed.sort_values(['부족한_주요시설수', '종합만족도'], ascending=[False, True])

print(f"\n=== 우선 개선 대상 시장 (상위 10개) ===")
display_cols = ['시장명_x', '종합만족도', '부족한_주요시설수'] + high_impact_facilities
display(improvement_needed_sorted[display_cols].head(10))

print(f"\n총 개선 대상 시장: {len(improvement_needed)}개")


In [ ]:
# 분석 결과를 CSV 파일로 저장
try:
    # 1. 전체 결합 데이터 저장
    df_merged.to_csv('전통시장_분석결과_전체.csv', index=False, encoding='utf-8-sig')
    print("✅ '전통시장_분석결과_전체.csv' 저장 완료")
    
    # 2. 분석 대상 데이터만 저장
    analysis_result = df_analysis[['시장명_x', '시도', '시군구', '종합만족도', '총시설수'] + facility_columns].copy()
    analysis_result.to_csv('전통시장_분석결과_요약.csv', index=False, encoding='utf-8-sig')
    print("✅ '전통시장_분석결과_요약.csv' 저장 완료")
    
    # 3. 개선 대상 시장 리스트 저장
    improvement_needed_sorted[display_cols].to_csv('개선대상_시장리스트.csv', index=False, encoding='utf-8-sig')
    print("✅ '개선대상_시장리스트.csv' 저장 완료")
    
    # 4. 시설별 영향도 분석 결과 저장
    summary_df.to_csv('시설별_만족도영향_분석결과.csv', index=False, encoding='utf-8-sig')
    print("✅ '시설별_만족도영향_분석결과.csv' 저장 완료")
    
    # 5. 시설별 리뷰 키워드 분석 결과 저장
    if not facility_keywords_df.empty:
        facility_keywords_df.to_csv('시설별_리뷰키워드_분석결과.csv', index=False, encoding='utf-8-sig')
        print("✅ '시설별_리뷰키워드_분석결과.csv' 저장 완료")
        file_count = 5
    else:
        file_count = 4
    
    # 6. 시설별 관심도-보유율 비교 분석 결과 저장
    if not comparison_df.empty:
        comparison_df.to_csv('시설별_관심도보유율_비교분석.csv', index=False, encoding='utf-8-sig')
        print("✅ '시설별_관심도보유율_비교분석.csv' 저장 완료")
        file_count += 1
    
    print(f"\n📁 총 {file_count}개의 결과 파일이 저장되었습니다.")
    
except Exception as e:
    print(f"❌ 파일 저장 중 오류 발생: {e}")

print("\n=== 분석 완료 ===")
print("🎉 전통시장 시설현황과 온라인 리뷰 데이터 분석이 성공적으로 완료되었습니다!")
print("📊 생성된 시각화와 결과 파일들을 활용하여 시설 개선 계획을 수립하시기 바랍니다.")
